## The Challenge

We are an asset manager who has just taken on a new client. The client has given us a number of mandates on how to manage their assets and we have assigned each of these to a different portfolio manager.

Even though there are a number of different mandates, some of the strategies that we are using are common across mandates. We really need to be able to see how each strategy is performing across all of our client's holdings, rather than just looking at a single portfolio in isolation.

## The Solution

Using LUSID we have the ability to add a strategy label to each transaction that we make. We can then aggregate across these labels to get a holistic view into how a strategy is performing. We can do this by:

1. Creating a portfolio for each mandate that our client has given us and group them in a portfolio group
2. Loading our instrument universe 
3. Adding our take on balances for each portfolio which have been provided to us by a transition manager
4. Creating a strategy tag that we can use to tag our transactions with the strategy used to make the trade
5. Adding the first trades that have been made by our portfolio managers inlcuding the strategy that they were following when they made them
6. Adding market data quotes so that we can aggregate across our client's portfolios
7. Aggregate across all of our client's portfolios to see the overall position
8. Aggregate across all of our client's portfolios by strategy to our position per strategy

First things first though, we need to initialise our LUSID environment. 

*To initialise our LUSID environment run the cell below*

In [1]:
# Import LUSID
import lusid.models as models
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')
print ('LUSID API Version: ', client.metadata.get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version:  0.5.2935.0


We now have an empty LUSID environment initialised

![Scopes](img/strategies-lusid.gif)

### 1) Creating a portfolio for each mandate and grouping them in a portfolio group

In LUSID we can create separate environments for each of our clients using scopes. A scope is a container for LUSID objects and can be thought of as a separate identity namespace. Using LUSID's entitlements engine we can ensure that only the right people can access a given scope. Furthermore we can ensure that only the right people can access the right objects inside a given scope. LUSID allows for entitlements down to the most granular level. 

For this example we will create a scope for our new client. Let us assume that our new client is a pension fund for a large high street retailer which we will call 'high_street_pension_fund'.

*Run the cell below to initialise our scope. Note that the scope will have a unique 4 character code appended to it to make the name more unique.*

In [2]:
# Fetch our scopes
scope_id = import_data.create_scope_id()
client_scope_code = 'client-high_street_pension_fund-{}'.format(scope_id)
# Pretty print our scope
prettyprint.heading('New Client Scope Code', client_scope_code)

New Client Scope Code: client-high_street_pension_fund-3794-e877-ae6f-2e


We now have a scope for our client inside our LUSID environment. 

![Scopes](img/strategies-scope.gif)

Now that we have our scope we can create our portfolios.

The first thing that we need to do here is give each portfolio a unique name. In LUSID these are refered to as 'codes'. Almost every object in LUSID can be refered to by the combination of the scope that it is contained in and its code. The code must be unique to the scope.

Our pension fund has five mandates and thus five portfolios. 

In LUSID we can use portfolio groups to group the different portfolios of each of our clients together. This allows us to easily keep track of and work with the holdings of each of our clients.

*Run the cell below to import the name for our portfolio group and its corresponding portfolios.*

In [3]:
client_portfolios = pd.read_csv('data/strategy-portfolios.csv')
client_portfolios.head()

,portfolio_group_name,portfolio_name
0,high_street_pension_fund-portfolios,mandate-balanced
1,high_street_pension_fund-portfolios,mandate-energy
2,high_street_pension_fund-portfolios,mandate-fixedincome
3,high_street_pension_fund-portfolios,mandate-international
4,high_street_pension_fund-portfolios,mandate-usgovt


Note that in LUSID we can only create one portfolio at a time. 
        
We will thus create each of the portfolios for our client one after the other and then we will create the portfolio group that binds them together.
        
We on-boarded this clients 5 days ago. When we create the portfolios we can set the creation date using the 'created' argument. This will allow us to backdate holdings and transactions.

You can read more about creating a portfolio in the LUSID API documentation: [LUSID API Docs: Creating a Portfolio](https://docs.lusid.com/#operation/CreatePortfolio)

You can also read more about creating a portfolio group in the LUSID API documentation:

[LUSID API Docs: Creating a Portfolio Group](https://docs.lusid.com/#operation/CreatePortfolioGroup)

*Run the cell below to create our portfolios and group them together*

In [4]:
# Set the creation date of our portfolio
created_date = datetime.now(pytz.UTC) - timedelta(days=5)

# Initialise a list to hold the ids of our portfolios for use in creating our portfolio group
portfolio_resourceids = []

# Loop over our portfolios
for portfolio_code in client_portfolios['portfolio_name'].unique():
    
    # Create the request to add our portfolio
    portfolio_request = models.CreateTransactionPortfolioRequest(
        display_name=portfolio_code,
        code=portfolio_code,
        base_currency='GBP',
        description=portfolio_code,
        created=created_date)
    
    # Call LUSID to create our portfolio
    response = client.transaction_portfolios.create_portfolio(
        scope=client_scope_code,
        transaction_portfolio=portfolio_request)

    # Print the response from LUSID using pretty formatting
    prettyprint.portfolio_response(response)

    # Add the portfolio to our list of portfolios for the portfolio group
    portfolio_resourceids.append(
        models.ResourceId(
            scope=client_scope_code, 
            code=portfolio_code)
    )

client_group_code = client_portfolios['portfolio_group_name'].unique()[0]
    
# Create our portfolio group request
portfolio_group_request = models.CreatePortfolioGroupRequest(
    code=client_group_code,
    display_name=client_group_code,
    values=portfolio_resourceids,
    description='Grouping all of {}'.format(client_group_code))

# Call LUSID to create our portfolio group
response = client.portfolio_groups.create_portfolio_group(
    scope=client_scope_code,
    request=portfolio_group_request)

# Print the response from LUSID using pretty formatting
prettyprint.portfolio_group_response(response, 'created')

Portfolio Created
Scope: client-high_street_pension_fund-3794-e877-ae6f-2e
Code: mandate-balanced
Portfolio Effective From: 2019-07-25 11:09:53.499253+00:00
Portfolio Created On: 2019-07-30 11:09:59.433097+00:00

Portfolio Created
Scope: client-high_street_pension_fund-3794-e877-ae6f-2e
Code: mandate-energy
Portfolio Effective From: 2019-07-25 11:09:53.499253+00:00
Portfolio Created On: 2019-07-30 11:10:00.771611+00:00

Portfolio Created
Scope: client-high_street_pension_fund-3794-e877-ae6f-2e
Code: mandate-fixedincome
Portfolio Effective From: 2019-07-25 11:09:53.499253+00:00
Portfolio Created On: 2019-07-30 11:10:01.128457+00:00

Portfolio Created
Scope: client-high_street_pension_fund-3794-e877-ae6f-2e
Code: mandate-international
Portfolio Effective From: 2019-07-25 11:09:53.499253+00:00
Portfolio Created On: 2019-07-30 11:10:05.673757+00:00

Portfolio Created
Scope: client-high_street_pension_fund-3794-e877-ae6f-2e
Code: mandate-usgovt
Portfolio Effective From: 2019-07-25 11:09:53.

Our scope for our new client is now populated with our portfolios which are grouped together in a portfolio group. All of our portfolios are currently empty.

![Scopes](img/strategies-portfolios.gif)

### 2) Adding our instrument universe

Before we can load our take on balances we first need to add securities to our instrument master in LUSID. 
        
The instrument master contains the details of all of the securities that we hold or trade in. Any time that we conduct a trade or add a holding of a security we refer to it in the instrument master. 

Below we can import our universe of instruments which all of our holdings are made up of. Using LUSID this universe can be pre-populated in advance with your entire instrument universe so that we don't have to keep adding new instruments when we make a trade or update a holding. 

*Run the cell below to import our instrument universe*

In [5]:
instrument_universe = pd.read_csv('data/instruments.csv')
instrument_universe.head(n=15)

,instrument_name,client_internal,currency,isin,figi,exchange_code,country_issue,ticker,market_sector,security_type,coupon
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,US0231351067,BBG000BVPXP1,UN,united_states_america,AMZN,equity,common_stock,NaN
1,Apple_Nasdaq_AAPL,imd_35345345,USD,US0378331005,BBG000B9XVV8,UN,united_states_america,AAPL,equity,common_stock,NaN
2,BP_LondonStockEx_BP,imd_43535553,GBP,GB0007980591,BBG000C05BD1,LN,united_kingdom,BP/,equity,common_stock,NaN
3,BurfordCapital_LondonStockEx_BUR,imd_43534356,GBP,GG00B4L84979,BBG000PN88Q7,LN,united_kingdom,BUR,equity,common_stock,NaN
4,EKFDiagnostics_LondonStockEx_EKF,imd_34535355,GBP,GB0031509804,BBG000BVNBN3,LN,united_kingdom,EKF,equity,common_stock,NaN
5,Glencore_LondonStockEx_GLEN,imd_34534555,GBP,JE00B4T3BW64,BBG001MM1KV4,LN,united_kingdom,GLEN,equity,common_stock,NaN
6,JustEat_LondonStockEx_JE,imd_35436366,GBP,GB00BKX5CN86,BBG0065YWM39,LN,united_kingdom,JE/,equity,common_stock,NaN
7,Kingfisher_LondonStockEx_KGF,imd_34535552,GBP,GB0033195214,BBG000BKH1W6,LN,united_kingdom,KGF,equity,common_stock,NaN
8,MicroFocus_LondonStockEx_MCRO,imd_34567338,GBP,GB00BD8YWM01,BBG000G4KKD2,LN,united_kingdom,MCRO,equity,common_stock,NaN
9,RELXGroup_LondonStockEx_REL,imd_43532542,GBP,GB00B2B0DG97,BBG000D03XD4,LN,united_kingdom,REL,equity,common_stock,NaN


Now that we have our instruments we can add them to LUSID.

To add securities to the instrument master in LUSID we use an upsert method. This method will insert a new record if it does not exist and update the existing record if it does exist. This allows us to add instruments more simply than have to check if an instrument exists before we insert it. 

We can upsert our instruments individually or in a batch. You can read more about upserting instruments
in the LUSID API Documentation: [LUSID API Docs: Upserting Instruments](https://docs.lusid.com/#operation/UpsertInstruments)

*Run the cell below to upsert our instrument universe*

In [6]:
# Initialise your batch upsert request
batch_upsert_request = {}

# Using your instrument universe create your batch request
for index, instrument in instrument_universe.iterrows():

    # Specify the columns of your identifiers
    identifier_columns = [
            ('isin', 'Isin'), 
            ('figi', 'Figi'), 
            ('ticker', 'Ticker'),
            ('client_internal', 'ClientInternal')
    ]
    
    # Create your identifiers
    identifiers = {}
    for identifier in identifier_columns:
        identifiers[identifier[1]] = models.InstrumentIdValue(
            value=instrument[identifier[0]])
    
    # Build your request and add it to the dictionary
    batch_upsert_request[instrument['instrument_name']] = models.InstrumentDefinition(
        name=instrument['instrument_name'],
        identifiers=identifiers)
    
# Call LUSID to upsert your instrument defintions
instrument_response = client.instruments.upsert_instruments(
    instruments=batch_upsert_request)

# Pretty print the response
prettyprint.instrument_response(instrument_response)

Instrument Successfully Upserted: MicroFocus_LondonStockEx_MCRO
ClientInternal ID: imd_34567338
LUSID Instrument ID: LUID_JX057SQ9


Instrument Successfully Upserted: USTreasury_2.00_2021
ClientInternal ID: imd_34535347
LUSID Instrument ID: LUID_5WSS6K9W


Instrument Successfully Upserted: Amazon_Nasdaq_AMZN
ClientInternal ID: imd_34634534
LUSID Instrument ID: LUID_ET747Z6X


Instrument Successfully Upserted: UKGiltTreasury_4.5_2034
ClientInternal ID: imd_34534534
LUSID Instrument ID: LUID_RF9W9YB4


Instrument Successfully Upserted: BurfordCapital_LondonStockEx_BUR
ClientInternal ID: imd_43534356
LUSID Instrument ID: LUID_VZPQ7S8P


Instrument Successfully Upserted: Sage_LondonStockEx_SGE
ClientInternal ID: imd_23536673
LUSID Instrument ID: LUID_S0H6O25U


Instrument Successfully Upserted: JustEat_LondonStockEx_JE
ClientInternal ID: imd_35436366
LUSID Instrument ID: LUID_NVEXJGA2


Instrument Successfully Upserted: EKFDiagnostics_LondonStockEx_EKF
ClientInternal ID: imd_34535355
LUSID

Our LUSID environment now has an instrument master containing all of our instruments that we will be trading.

![Scopes](img/strategies-instrumentmaster.gif)

### 3) Adding our take on balances for each portfolio which have been provided to us by a transition manager

We have been advised by a transition manager on the holdings that we will be taking on for our new client. They have provided these initial holdings to us in a CSV.

Let us load our take on balances. 

*Run the cell below to import our initial holdings information*

In [7]:
client_holdings = pd.read_csv('data/strategy-holdings.csv')
client_holdings.head(n=20)

,portfolio_group_name,portfolio_name,instrument_name,quantity,price,currency,figi
0,high_street_pension_fund-portfolios,mandate-balanced,Kingfisher_LondonStockEx_KGF,1362038,2.276,GBP,BBG000BKH1W6
1,high_street_pension_fund-portfolios,mandate-balanced,JustEat_LondonStockEx_JE,834553,5.464,GBP,BBG0065YWM39
2,high_street_pension_fund-portfolios,mandate-balanced,RELXGroup_LondonStockEx_REL,494343,15.980,GBP,BBG000D03XD4
3,high_street_pension_fund-portfolios,mandate-balanced,UKGiltTreasury_4.5_2034,77481,140.572,GBP,BBG0000D14P3
4,high_street_pension_fund-portfolios,mandate-balanced,GBP_Cash,952000,1.000,GBP,NaN
5,high_street_pension_fund-portfolios,mandate-energy,Glencore_LondonStockEx_GLEN,905141,2.762,GBP,BBG001MM1KV4
6,high_street_pension_fund-portfolios,mandate-energy,BP_LondonStockEx_BP,1713922,5.114,GBP,BBG000C05BD1
7,high_street_pension_fund-portfolios,mandate-energy,GBP_Cash,2200000,1.000,GBP,NaN
8,high_street_pension_fund-portfolios,mandate-fixedincome,UKGiltTreasury_3.5_2045,266169,134.433,GBP,BBG006N6HZM7
9,high_street_pension_fund-portfolios,mandate-fixedincome,UKGiltTreasury_2.0_2025,405589,106.637,GBP,BBG0088JSC32


To set our holdings in LUSID we use the set holdings method. This method takes a list of adjustment requests. Using these requests LUSID will determine what transactions are required to update our portfolios so that they have the desired holdings.

We will make our holdings effecitve as of five days ago as this was when we on-boarded the client. 

Just like when we created our portfolios we can only set the holdings one portfolio at a time. 

You can read more about setting the holdings on a portfolio in the LUSID API Documentation: [LUSID API Docs: Setting the Holdings on a Portfolio](https://docs.lusid.com/#operation/SetHoldings)

*Run the cell below to set our take on balances*

In [8]:
holdings_effective_date = (datetime.now(pytz.UTC) - timedelta(days=5))

# Iterate over our portfolios
for portfolio_name in client_holdings['portfolio_name'].unique():
    
    # Initialise our list to hold the holding adjustments
    holding_adjustments = []

    # Iterate over the holdings in each portfolio
    for index, holding in client_holdings.loc[client_holdings['portfolio_name'] == portfolio_name].iterrows():
        
        # Get the currency of the holding
        currency = holding['currency']
        
        # Identify and handle cash instruments
        if 'Cash' in holding['instrument_name']:
            identifier_key = 'Instrument/default/Currency'
            identifier = currency
        # Else treat it as a normal instrument and use Figi
        else:
            identifier_key = 'Instrument/default/Figi'
            identifier = holding['figi']

        # Create our adjust holdings request using our instrument universe to get the LUID identifier for the instrument
        holding_adjustments.append(
            models.AdjustHoldingRequest(
                instrument_identifiers={
                    identifier_key: identifier},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=holding['quantity'],
                        cost=models.CurrencyAndAmount(
                            amount=holding['quantity'] * holding['price'],
                            currency=currency),
                        portfolio_cost=holding['quantity'] * holding['price'],
                        price=holding['price'])
                ])
        )
    
    # Call LUSID to set our holdings
    set_holdings_response = client.transaction_portfolios.set_holdings(
        scope=client_scope_code,
        code=portfolio_name,
        effective_at=holdings_effective_date,
        holding_adjustments=holding_adjustments)
    
    # Print the response from LUSID using pretty formatting
    prettyprint.set_holdings_response(
        set_holdings_response, 
        client_scope_code, 
        portfolio_name)

print ('Holdings have been set for all Portfolios')

Holdings Successfully Set for Portfolio
Scope: client-high_street_pension_fund-3794-e877-ae6f-2e
Code: mandate-balanced
Holdings Effective From: 2019-07-25 11:10:13.577335+00:00
Holdings Created On: 2019-07-30 11:10:21.075115+00:00

Holdings Successfully Set for Portfolio
Scope: client-high_street_pension_fund-3794-e877-ae6f-2e
Code: mandate-energy
Holdings Effective From: 2019-07-25 11:10:13.577335+00:00
Holdings Created On: 2019-07-30 11:10:27.567202+00:00

Holdings Successfully Set for Portfolio
Scope: client-high_street_pension_fund-3794-e877-ae6f-2e
Code: mandate-fixedincome
Holdings Effective From: 2019-07-25 11:10:13.577335+00:00
Holdings Created On: 2019-07-30 11:10:28.287410+00:00

Holdings Successfully Set for Portfolio
Scope: client-high_street_pension_fund-3794-e877-ae6f-2e
Code: mandate-international
Holdings Effective From: 2019-07-25 11:10:13.577335+00:00
Holdings Created On: 2019-07-30 11:10:28.630468+00:00

Holdings Successfully Set for Portfolio
Scope: client-high_str

Our portfolios are no longer empty and now contain the initial holdings for our new client.

![Scopes](img/strategies-holdings.gif)

### 4) Creating a strategy tag that we can use to tag our transactions with the strategy used to make the trade

We want to be able to track what strategies our portfolio managers were following when they made an investment decision. To do this in LUSID we can use properties. We can define a property and attach it to almost any object in LUSID. 

In this case we can create a 'strategy' property that we can attach to our transactions. This property will take the form of a string which will be the name of the strategy that the portfolio manager followed. Examples of strategies in this case are 'quantitativeSignal', 'fundamentalAnalysis' etc.

You can read more about defining new properties in the LUSID API documentation: [LUSID API Docs: Creating a New Property Definition](https://docs.lusid.com/#operation/CreatePropertyDefinition)

*Run the cell below to create our strategy property*

In [9]:
# Create our request to define a new property
property_request = models.CreatePropertyDefinitionRequest(
    domain='Transaction',
    scope=client_scope_code,
    code='strategy',
    value_required=False,
    display_name='strategy',
    data_type_id=models.ResourceId(scope='system', code='string'))

# Call LUSID to create our new property
property_response = client.property_definitions.create_property_definition(
    definition=property_request)

# Grab the key off the response to use when referencing this property in other LUSID calls
strategy_property_key = property_response.key

# Pretty print our key
prettyprint.heading('Strategy Property Key', strategy_property_key)

Strategy Property Key: Transaction/client-high_street_pension_fund-3794-e877-ae6f-2e/strategy


We now have a strategy property that we can set on each of our transactions which is referenced by our strategy property key.

![Scopes](img/strategies-strategypropertykey.gif)

### 5) Adding the first trades that have been made by our portfolio managers inlcuding the strategy that they were following when they made them

Now that we have our transaction property defined we can load our transactions. We will load these from a CSV file. In practice these are likely to come from our order management system. You can see from looking at the transaction dates that these transactions have all occured over the last 4 days.

*Run the cell below to import our transactions*

In [10]:
client_transactions = import_data.fetch_client_transactions(
    'strategy-transactions.csv', 
    days_back=4)

client_transactions.head(n=10)

,transaction_id,type,portfolio,instrument_name,instrument_uid,transaction_date,settlement_date,units,transaction_price,transaction_currency,total_cost,strategy,description
0,tid_329432525234324,Sell,mandate-balanced,Kingfisher_LondonStockEx_KGF,BBG000BKH1W6,2019-07-29T09:57:41.993150+00:00,2019-07-31T09:57:41.993150+00:00,325000,2.345,GBP,762125.000,quantitativeSignal,Equity Sale
1,tid_325452342424500,Buy,mandate-balanced,UKGiltTreasury_4.5_2034,BBG0000D14P3,2019-07-29T10:30:26.615165+00:00,2019-07-31T10:30:26.615165+00:00,10501,140.572,GBP,1476146.572,incomeRequirements,Equity Purchase
2,tid_234295929052090,Buy,mandate-fixedincome,UKGiltTreasury_3.75_2021,BBG001KKJLR4,2019-07-29T09:40:56.730522+00:00,2019-07-31T09:40:56.730522+00:00,24000,109.126,GBP,2619024.000,incomeRequirements,Equity Purchase
3,tid_234942982496001,Sell,mandate-fixedincome,USTreasury_2.00_2021,BBG00FN3B5K8,2019-07-28T16:15:30.868340+00:00,2019-07-30T16:15:30.868340+00:00,57000,97.800,USD,5574600.000,internationalExposure,Equity Sale
4,tid_239491298599900,Sell,mandate-energy,Glencore_LondonStockEx_GLEN,BBG001MM1KV4,2019-07-27T13:14:41.077202+00:00,2019-07-29T13:14:41.077202+00:00,120000,2.600,GBP,312000.000,fundamentalAnalysis,Equity Sale
5,tid_121309590059995,Buy,mandate-energy,BP_LondonStockEx_BP,BBG000C05BD1,2019-07-29T16:32:21.321453+00:00,2019-07-31T16:32:21.321453+00:00,50000,5.287,GBP,264350.000,fundamentalAnalysis,Equity Purchase
6,tid_124319009950924,Sell,mandate-usgovt,USTreasury_2.00_2021,BBG00FN3B5K8,2019-07-27T10:02:24.158341+00:00,2019-07-29T10:02:24.158341+00:00,28000,98.100,USD,2746800.000,incomeRequirements,Equity Sale
7,tid_212390582999593,Buy,mandate-international,Apple_Nasdaq_AAPL,BBG000B9XVV8,2019-07-28T16:23:16.507857+00:00,2019-07-30T16:23:16.507857+00:00,15000,168.490,GBP,2527350.000,newsSentiment,Equity Purchase


Now that we have our transactions we can upsert them into LUSID. We can only upsert transactions for one portfolio at time. Therefore we will upsert our transactions in batches. Each batch contains all the transactions for a given portfolio. 

You can read more about upserting transactions in the LUSID API documentation: [LUSID API Docs: Upserting Transactions](https://docs.lusid.com/#operation/UpsertTransactions)

*Run the cell below to upsert our transactions*

In [11]:
# Iterate over our portfolios
for portfolio_name in client_transactions['portfolio'].unique():
    # Initialise a list to hold our transactions for each portfolio
    batch_transaction_requests = []
    # Iterate over the transactions for each portfolio
    for index, transaction in client_transactions.loc[
        client_transactions['portfolio'] == portfolio_name].iterrows():
        # Append the transaction to our request, note the use of the 'properties' argument
        batch_transaction_requests.append(
            models.TransactionRequest(
                transaction_id=transaction['transaction_id'],
                type=transaction['type'],
                instrument_identifiers={
                    'Instrument/default/Figi': transaction['instrument_uid']},
                transaction_date=transaction['transaction_date'],
                settlement_date=transaction['settlement_date'],
                units=transaction['units'],
                transaction_price=models.TransactionPrice(
                  price=transaction['transaction_price'],
                  type='Price'),
                total_consideration=models.CurrencyAndAmount(
                  amount=transaction['total_cost'],
                  currency=transaction['transaction_currency']),
                 source='Client',
                 transaction_currency=transaction['transaction_currency'],
                 properties={
                     strategy_property_key: models.PerpetualProperty(
                         key=strategy_property_key,
                         value=models.PropertyValue(label_value=transaction['strategy'])
                     )
                 }
              ))
            
    # Call LUSID to upsert our transactions
    transaction_response = client.transaction_portfolios.upsert_transactions(
        scope=client_scope_code,
        code=portfolio_name,
        transactions=batch_transaction_requests)
    
    # Print the response from LUSID using pretty formatting 
    prettyprint.transactions_response(
        transaction_response,
        client_scope_code,
        portfolio_name)

Transactions Successfully Upserted into Portfolio
Scope: client-high_street_pension_fund-3794-e877-ae6f-2e
Code: mandate-balanced
Transactions Effective From: 2019-07-29 10:30:26.615165+00:00
Transactions Created On: 2019-07-30 11:10:33.013813+00:00

Transactions Successfully Upserted into Portfolio
Scope: client-high_street_pension_fund-3794-e877-ae6f-2e
Code: mandate-fixedincome
Transactions Effective From: 2019-07-29 09:40:56.730522+00:00
Transactions Created On: 2019-07-30 11:10:33.286206+00:00

Transactions Successfully Upserted into Portfolio
Scope: client-high_street_pension_fund-3794-e877-ae6f-2e
Code: mandate-energy
Transactions Effective From: 2019-07-29 16:32:21.321453+00:00
Transactions Created On: 2019-07-30 11:10:34.481467+00:00

Transactions Successfully Upserted into Portfolio
Scope: client-high_street_pension_fund-3794-e877-ae6f-2e
Code: mandate-usgovt
Transactions Effective From: 2019-07-27 10:02:24.158341+00:00
Transactions Created On: 2019-07-30 11:10:34.758937+00:0

Now each of our portfolios contains a list of transactions, the first are 'StockIn' transactions generated by LUSID to set our holdings, the second set are the transactions from our portfolio managers which are tagged with a strategy.

![Scopes](img/strategies-transactions.gif)

### 6) Adding market data quotes so that we can aggregate across our client's portfolios

Now that we have populated our portfolios we want to aggregate across them to see both our client's overall position as well as our exposure as the result of each strategy that our portfolio managers are using. 

To this we first need to add some market data to the LUSID quote store.

You can read more about quotes in the LUSID API documentation: [LUSID API Docs: Quotes](https://support.lusid.com/what-is-a-quote)

*Run the cell below to create add our quotes*

In [12]:
# Set our analytics effective date to be from now
quotes_effective_date = datetime.now(pytz.UTC)
instrument_quotes = {}

# Create dummy prices of 1 for all instruments except cash which we will ignore
for index, instrument in instrument_universe.iterrows():
    
    if 'Cash' in instrument['instrument_name']:
        continue
    
    luid = client.instruments.get_instrument(
        identifier_type='Figi',
        identifier=instrument['figi']).lusid_instrument_id
    
    instrument_quotes[luid] = models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                quote_series_id=models.QuoteSeriesId(
                    provider='Lusid',
                    instrument_id=luid,
                    instrument_id_type='LusidInstrumentId',
                    quote_type='Price',
                    field='Mid'),
                effective_at=quotes_effective_date
            ),
            metric_value=models.MetricValue(
                value=1,
                unit="GBP"),
            lineage='InternalSystem')

# Upsert the quotes into LUSID
response = client.quotes.upsert_quotes(
    scope=client_scope_code,
    quotes=instrument_quotes)

# Pretty print the response from LUSID
prettyprint.upsert_quotes_response(response)

Analytic Store Created
Scope: client-high_street_pension_fund-3794-e877-ae6f-2e
Effective From: 2019-07-30T00:00:00+00:00


We now have market data in our new client's scope that we can use for our aggregation requests.

![Scopes](img/strategies-analyticsstore.gif)

### 7) Aggregate across all of our client's portfolios to see the overall position

The first thing we want to do is aggregate across our client's entire group of portfolios to see their overall poistion. We can do this by aggregating the holdings by the portfolio group. We will sum the units and the cost of each instrument across the group.

You can read more about creating by portfolio group in the LUSID API documentation: [LUSID API Docs: Aggregate by Portfolio Group](https://docs.lusid.com/#operation/GetAggregationByGroup)

*Run the cell below to aggregate across all of our client's portfolios*

In [14]:
# Create our aggregation request
inline_recipe = models.ConfigurationRecipe(
    code='quotes_recipe',
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
               key='Equity.LusidInstrumentId.*',
               supplier='Lusid',
               data_scope=client_scope_code,
               quote_type='Price',
               field='Mid')
        ],
        suppliers=models.MarketContextSuppliers(
            commodity='Lusid',
            credit='Lusid',
            equity='Lusid',
            fx='Lusid',
            rates='Lusid'),
        options=models.MarketOptions(
            default_supplier='Lusid',
            default_instrument_code_type='LusidInstrumentId',
            default_scope=client_scope_code)
        )
    )

aggregation_request = models.AggregationRequest(
    inline_recipe=inline_recipe,
    effective_at=datetime.now(pytz.UTC),
    metrics=[
        models.AggregateSpec(key='Holding/default/SubHoldingKey',
        op='Value'),
        models.AggregateSpec(key='Instrument/default/Name',
        op='Value'),
        models.AggregateSpec(key='Holding/default/Units',
        op='Sum'),
        models.AggregateSpec(key='Holding/default/Cost',
        op='Sum')
    ],
    group_by=[
        'Holding/default/SubHoldingKey'
    ])

# Call LUSID to aggregate across all of our portfolios
aggregated_group = client.aggregation.get_aggregation_by_group(
    scope=client_scope_code,
    code=client_group_code,
    request=aggregation_request)

# Pretty print our response from LUSID
prettyprint.aggregation_response(aggregated_group)

LUSID Instrument Id / Currency: LUID_ROVUL6FT
Instrument Name: Kingfisher_LondonStockEx_KGF
Instrument Units: 1037038.0
Instrument Cost: 2360298.49
Instrument Currency: GBP

LUSID Instrument Id / Currency: LUID_NVEXJGA2
Instrument Name: JustEat_LondonStockEx_JE
Instrument Units: 834553.0
Instrument Cost: 4559997.59
Instrument Currency: GBP

LUSID Instrument Id / Currency: LUID_OVPUXTI2
Instrument Name: RELXGroup_LondonStockEx_REL
Instrument Units: 494343.0
Instrument Cost: 7899601.14
Instrument Currency: GBP

LUSID Instrument Id / Currency: LUID_RF9W9YB4
Instrument Name: UKGiltTreasury_4.5_2034
Instrument Units: 87982.0
Instrument Cost: 12367805.7
Instrument Currency: GBP

LUSID Instrument Id / Currency: GBP
Instrument Name: GBP
Instrument Units: 789254.43
Instrument Cost: 789254.43
Instrument Currency: GBP

LUSID Instrument Id / Currency: LUID_ZA0IFTG5
Instrument Name: Glencore_LondonStockEx_GLEN
Instrument Units: 785141.0
Instrument Cost: 2168559.44
Instrument Currency: GBP

LUSID In

Now we have a view of our client's holdings across their entire group of portfolios.

![Scopes](img/strategies-aggregatedportfoliogroup.gif)

### 8) Aggregate across all of our client's portfolios by strategy to our position per strategy

Now that we have a view of our client's overall position we want to get a sense as to what strategies our portfolio manager's have been using to make their trades and how our position looks by strategy.

To do this we will make us of Derived Portfolios. 

A derived portfolio is a portfolio that is derived from a parent portfolio. It inherits all of its transactions from the parent portfolio. We can make use of derived portfolios to apply sub-holding keys to our holdings. In this case we will use our transaction strategy property as our sub-holding key.

We will create a new scope to hold our derived portfolios. This will allow us to give them the same name as the parent portfolio.

*Run the cell below to create our new scope*

In [15]:
# Fetch our scopes
scope_id = import_data.create_scope_id()
client_strategy_scope_code = 'client-strategy-high_street_pension_fund-{}'.format(scope_id)
# Pretty print our scope
prettyprint.heading('New Client Strategy Scope Code', client_strategy_scope_code)

New Client Strategy Scope Code: client-strategy-high_street_pension_fund-3794-e8a3-70de-8a


Now that we have our new scope we can create our derived portfolios. We will create one for each of the corresponding parent portfolios. We will also group our derived portfolios together so that we can aggregate across the group.

Note that we use the same creation date for the derived portfolios as we did for the parent portfolio. 

You can read more about creating a derived portfolio in the LUSID API documentation: [LUSID API Docs: Creating a Derived Portfolio](https://docs.lusid.com/#operation/CreateDerivedPortfolio)

*Run the cell below to create and group our derived portfolios*

In [16]:
# Initialise a list to hold the ids of our portfolios for use in creating our portfolio group
portfolio_resourceids = []

# Loop over our portfolios
for portfolio_code in client_portfolios['portfolio_name'].unique():
    
    # Create the request to add our portfolio
    portfolio_request = models.CreateDerivedTransactionPortfolioRequest(
        display_name=portfolio_code,
        code=portfolio_code,
        parent_portfolio_id=models.ResourceId(
            scope=client_scope_code,
            code=portfolio_code),
        description=portfolio_code,
        created=created_date,
        sub_holding_keys=[strategy_property_key])
    
    # Call LUSID to create our portfolio
    portfolio_response = client.derived_transaction_portfolios.create_derived_portfolio(
        scope=client_strategy_scope_code,
        portfolio=portfolio_request)
    
    # Print the response from LUSID using pretty formatting
    prettyprint.portfolio_response(portfolio_response)

    # Add the portfolio to our list of portfolios for the portfolio group
    portfolio_resourceids.append(
        models.ResourceId(
            scope=client_strategy_scope_code, 
            code=portfolio_code))

# Create our portfolio group request
portfolio_group_request = models.CreatePortfolioGroupRequest(
    code=client_group_code,
    display_name=client_group_code,
    values=portfolio_resourceids,
    description='Grouping all of {}'.format(client_group_code))

# Call LUSID to create our portfolio group
response = client.portfolio_groups.create_portfolio_group(
    scope=client_strategy_scope_code,
    request=portfolio_group_request)

# Print the response from LUSID using pretty formatting
prettyprint.portfolio_group_response(response, 'created')

Derived Portfolio Created
Scope: client-strategy-high_street_pension_fund-3794-e8a3-70de-8a
Code: mandate-balanced
Portfolio Effective From: 2019-07-25 11:09:53.499253+00:00
Portfolio Created On: 2019-07-30 11:11:08.708238+00:00

   Parent Portfolio Details
   Scope: client-high_street_pension_fund-3794-e877-ae6f-2e
   Code: mandate-balanced

Derived Portfolio Created
Scope: client-strategy-high_street_pension_fund-3794-e8a3-70de-8a
Code: mandate-energy
Portfolio Effective From: 2019-07-25 11:09:53.499253+00:00
Portfolio Created On: 2019-07-30 11:11:09.229860+00:00

   Parent Portfolio Details
   Scope: client-high_street_pension_fund-3794-e877-ae6f-2e
   Code: mandate-energy

Derived Portfolio Created
Scope: client-strategy-high_street_pension_fund-3794-e8a3-70de-8a
Code: mandate-fixedincome
Portfolio Effective From: 2019-07-25 11:09:53.499253+00:00
Portfolio Created On: 2019-07-30 11:11:09.663904+00:00

   Parent Portfolio Details
   Scope: client-high_street_pension_fund-3794-e877-a

![Scopes](img/strategies-derived.gif)

Now that we have created our derived portfolios we can aggregate across our group of derived portfolios. Note that we can use the market data quotes from the previous aggregation in our main scope as we only had dummy prices in there.

*Run the cell below to aggregate by strategy across our client's holdings*

In [17]:
aggregation_request = models.AggregationRequest(
    inline_recipe=inline_recipe,
    effective_at=datetime.now(pytz.UTC),
    metrics=[
        models.AggregateSpec(
            key=strategy_property_key,
            op='Value'),
        models.AggregateSpec(
            key='Holding/default/Cost',
            op='Sum')],
    group_by=[
        strategy_property_key],
    filters=[
        models.PropertyFilter(
            left=strategy_property_key,
            operator='NotEquals',
            right='<Not Classified>',
            right_operand_type='Absolute'),
        models.PropertyFilter(
            left="Instrument/default/Name",
            operator='NotEquals',
            right='GBP',
            right_operand_type='Absolute'),
        models.PropertyFilter(
            left="Instrument/default/Name",
            operator='NotEquals',
            right='USD',
            right_operand_type='Absolute')
    ]
    )

aggregated_group = client.aggregation.get_aggregation_by_group(
    scope=client_strategy_scope_code,
    code=client_group_code,
    request=aggregation_request)

# Pretty print our response from LUSID
prettyprint.aggregation_response_strategy(aggregated_group, strategy_property_key)

Strategy: quantitativeSignal
Strategy Cost: -762125.0

Strategy: incomeRequirements
Strategy Cost: 1348370.57

Strategy: fundamentalAnalysis
Strategy Cost: -47650.0

Strategy: internationalExposure
Strategy Cost: -5574600.0

Strategy: newsSentiment
Strategy Cost: 2527350.0



We can now see how much we have invested into each of the different investment strategies. Our final state looks like the below.

![Scopes](img/strategies-strategyaggregation.gif)